# 1. Cargue sus datos

Por favor cargue los datos en la siguiente sección

## 1.1 Objetivos

- Objetivo general
- Objetivos especificos

## 1.2 Hipótesis

# 1.3. ¿Qué pregunta(s) quiere responder a partir de sus datos?

El proyecto de este modulo es muy amplio, debido a las diferentes arquitecturas de bases de datos presentadas y a sus deseos.

- Sqlite

- Postgresql

- Cassandra

- MongoDB

- Dask

- Spark

# Sqlite
---

- Cargar datos de distinta naturaleza (4 o 5 tablas), con interrelaciones
- Distintos tipos de consulta



### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en sqlite y enviarlos a mongodb o a PySpark a un RDD
- Realizar al menos unas 3 o 4 consultas distintas utilizando `common table expressions (CTEs) with o subqueries`
- Crear indices en distintas tablas explicando porque son necesarios

### Avanzados
- Enviar datos a Apache Kafka y serializarlos a sqlite utilizando Python
- Cargar datos en sqlite y hacer machine learning en Dask o PySpark utilizando archivos parquet o csv

# Postgresql (postgis análisis geoespacial y qgis)
---

- Cargar datos de distinta naturaleza
- Distintios tipos de consulta
- Clara definición de datos
- Resolver la hipótesis que se esta planteando

## Ejemplos


### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en postgresql y enviarlos a mongodb o a PySpark a un RDD
- Cargar datos en formato JSONB en postgresql
- Crear indices en distintas tablas explicando porque son necesarios
- Hacer consultas en formato JSONB en postgresql
- Realizar al menos unas 3 o 4 consultas distintas utilizando `common table expressions (CTEs)` o subqueries

### Avanzados
- Crear un ambiente principal-secundario en postgresql
- Enviar datos a Kafka y serializarlos a postgresql utilizando Python
- Cargar datos en Postgresql y hacer machine learning en Dask o PySpark

# MongoDB
---

- Realizar distintos tipos de agregación
- Realizar distintos tipos de agrupamiento
- Plantear el modelo `query first design`
-


## Ejemplos

### Sencillos
- Cargar datos
- Análisis de datos
- Consultas básicas
- Consultas avanzadas

### Medios
- Cargar datos en postgresql y enviarlos a mongodb
- Cargar datos en MongoDB y procesarlos usando Dask o usando PySpark
- Realizar al menos unas 3 o 4 consultas distintas utilizando map-reduce

### Avanzados
- Docker para hacer un ambiente distribuido
- Con RaspberryPI crear un cluster de datos
- Cargar datos en MongoDB y hacer machine learning en Dask o PySpark

# Dask
---


## Ejemplos

### Sencillos
- Cargar datos de csv, json, xml, etc.
- Realizar un procesamiento distribuido utilizan n-jobs
- Crear un par de consultas que involucren particiones de datos con un grafo de al menos dos o tres niveles

### Medios
- Cargar datos en postgresql y procesarlos usando dask.
- Cargar datos en MongoDB y procesarlos usando Dask
- Comparar los resultados de hacer machine learning con SkLearn vs hacerlo con Dask-ML

### Avanzados
- Docker para hacer un ambiente distribuido
- Con RaspberryPI crear un cluster de datos y analizar los datos con Dask
- Cargar datos en MongoDB y hacer machine learning en Dask

In [7]:
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import psutil
from pprint import pprint
import pymongo


from dask_mongo import read_mongo, to_mongo

## Conexión a la base de datos de Mongo

In [8]:
client = pymongo.MongoClient("mongodb://localhost:27017")
bd= client["Proyecto_MOD_3"]
collection_Test = bd["Test"]
collection_Train = bd["Train"]


Muestra de la base

In [9]:
pprint(collection_Test.find_one({}))

{'_id': ObjectId('648f8586aec8990bd2ff8e99'),
 'channelGrouping': 'Organic Search',
 'customDimensions': "[{'index': '4', 'value': 'APAC'}]",
 'date': 20180511,
 'device': {'browser': 'Chrome',
            'browserSize': 'not available in demo dataset',
            'browserVersion': 'not available in demo dataset',
            'deviceCategory': 'mobile',
            'flashVersion': 'not available in demo dataset',
            'isMobile': True,
            'language': 'not available in demo dataset',
            'mobileDeviceBranding': 'not available in demo dataset',
            'mobileDeviceInfo': 'not available in demo dataset',
            'mobileDeviceMarketingName': 'not available in demo dataset',
            'mobileDeviceModel': 'not available in demo dataset',
            'mobileInputSelector': 'not available in demo dataset',
            'operatingSystem': 'Android',
            'operatingSystemVersion': 'not available in demo dataset',
            'screenColors': 'not availab

In [10]:
pprint(collection_Train.find_one({}))

{'_id': ObjectId('648f86efaec8990bd205af51'),
 'channelGrouping': 'Organic Search',
 'customDimensions': "[{'index': '4', 'value': 'EMEA'}]",
 'date': 20171016,
 'device': {'browser': 'Firefox',
            'browserSize': 'not available in demo dataset',
            'browserVersion': 'not available in demo dataset',
            'deviceCategory': 'desktop',
            'flashVersion': 'not available in demo dataset',
            'isMobile': False,
            'language': 'not available in demo dataset',
            'mobileDeviceBranding': 'not available in demo dataset',
            'mobileDeviceInfo': 'not available in demo dataset',
            'mobileDeviceMarketingName': 'not available in demo dataset',
            'mobileDeviceModel': 'not available in demo dataset',
            'mobileInputSelector': 'not available in demo dataset',
            'operatingSystem': 'Windows',
            'operatingSystemVersion': 'not available in demo dataset',
            'screenColors': 'not avai

## Limpieza de datos

Cantidad de documentos con 'customDimensions' = []

In [11]:
N_doc_Test = collection_Test.count_documents({})
N_doc_Train = collection_Train.count_documents({})

print(f"""Base de datos conformada por:\n {N_doc_Test} para Test \n {N_doc_Train} para Train""")

Base de datos conformada por:
 401589 para Test 
 1708337 para Train


Dado que hay campos con estructura JSON pero tomandos como lista:

In [12]:
pprint(collection_Test.find_one({}, {"customDimensions":1,"hits":1}))

{'_id': ObjectId('648f8586aec8990bd2ff8e99'),
 'customDimensions': "[{'index': '4', 'value': 'APAC'}]",
 'hits': "[{'hitNumber': '1', 'time': '0', 'hour': '21', 'minute': '29', "
         "'isInteraction': True, 'page': {'pagePath': '/home', 'hostname': "
         "'shop.googlemerchandisestore.com', 'pageTitle': 'Home', "
         "'pagePathLevel1': '/home', 'pagePathLevel2': '', 'pagePathLevel3': "
         "'', 'pagePathLevel4': ''}, 'appInfo': {'screenName': "
         "'shop.googlemerchandisestore.com/home', 'landingScreenName': "
         "'shop.googlemerchandisestore.com/home', 'exitScreenName': "
         "'shop.googlemerchandisestore.com/home', 'screenDepth': '0'}, "
         "'exceptionInfo': {'isFatal': True}, 'eventInfo': {'eventCategory': "
         "'Enhanced Ecommerce', 'eventAction': 'Promotion Click'}, 'product': "
         "[], 'promotion': [{'promoId': 'Category Row 2', 'promoName': "
         "'Accessories', 'promoCreative': 'toy.png', 'promoPosition': "
         "'C

Se revisan valores vacíos (No poseen Json dentro de la lista)

In [13]:
def Vacios_json_list(field_name, collection):
    query = {field_name: []}
    count = collection.count_documents(query)
    return count

field_name = 'customDimensions'
print(f"Vacíos en {field_name} de Test: ",Vacios_json_list(field_name, collection_Test))
print(f"Vacíos en {field_name} de Train: ",Vacios_json_list(field_name, collection_Train))

field_name = 'hits'
print(f"Vacíos en {field_name} de Test: ",Vacios_json_list(field_name, collection_Test))
print(f"Vacíos en {field_name} de Train: ",Vacios_json_list(field_name, collection_Train))

Vacíos en customDimensions de Test:  60581
Vacíos en customDimensions de Train:  333235
Vacíos en hits de Test:  58
Vacíos en hits de Train:  1215


In [15]:
query = {"customDimensions" : {'$eq': []},
         "hits" : {'$eq': []}}

collection_Test.delete_many(query)
collection_Train.delete_many(query)

In [17]:
client.close()

## Uso del dataset en Dask

In [23]:

base_Test = read_mongo(
    connection_kwargs={"host": "mongodb://localhost:27017"},
    database="Proyecto_MOD_3",
    collection="Test",
    chunksize=500,
)

base_Train = read_mongo(
    connection_kwargs={"host": "mongodb://localhost:27017"},
    database="Proyecto_MOD_3",
    collection="Train",
    chunksize=500,
)



In [24]:
base_Test.pluck('channelGrouping').frequencies().compute()

[('Organic Search', 198378),
 ('Direct', 76076),
 ('Affiliates', 10833),
 ('Referral', 59498),
 ('Paid Search', 12834),
 ('Social', 36881),
 ('Display', 7076),
 ('(Other)', 6)]

In [25]:
base_Train.pluck('channelGrouping').frequencies().compute()

[('Organic Search', 738963),
 ('Referral', 211269),
 ('Direct', 273134),
 ('Paid Search', 45627),
 ('Display', 51283),
 ('Affiliates', 32915),
 ('Social', 354971),
 ('(Other)', 137)]